The NHS dm+d has a flag to indicate as to whether this item relates to a pack that is only to be made available through
hospital prescribing [(see tech spec of dm+d)](https://www.nhsbsa.nhs.uk/sites/default/files/2017-02/Data_Model_R2_v3.1_May_2015.pdf). 

Unusually the information appears to be held at AMP and  AMPP level. It is held in "Restrictions on availability" at AMP level and in the "product prescribing information" at AMPP level. This notebook explores what prescribing is going on in primary care for the possibility of a new measure.

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
##ensure £ and pence are set right
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### AMP LEVEL

In [3]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  Practice_Name, 
  Practice_Code,
  presc.bnf_code,
  dmd.id,
  avail_restrict,
  bnf_description AS bnf_name,
  Items,
  Quantity,
  Actual_Cost
FROM
  ebmdatalab.tmp_eu.raw_prescribing_data_2019_04 AS presc ##this is raw quantity file. using this to see if there are patterns of small quantities
Left JOIN  
  ebmdatalab.dmd.amp AS dmd
ON
presc.bnf_code = dmd.bnf_code
WHERE
avail_restrict = 8 ## this is code for hospital Rx
'''

df_amp = bq.cached_read(sql, csv_path='hospamp.csv')
df_amp.rename(columns={'PCO_Code':'pct'}, inplace=True)
df_amp.head(5)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,PCO_Name,pct,Practice_Name,Practice_Code,bnf_code,id,avail_restrict,bnf_name,Items,Quantity,Actual_Cost
0,EASTERN CHESHIRE CCG,01C00,MIDDLEWOOD PARTNERSHIP,N81022,0211000P0AAAAAA,22451411000001103,8,Tranexamic Acid_Inj 100mg/ml 5ml Amp,2,5,13.96
1,EASTERN CHESHIRE CCG,01C00,MIDDLEWOOD PARTNERSHIP,N81022,0211000P0AAAAAA,22786211000001105,8,Tranexamic Acid_Inj 100mg/ml 5ml Amp,2,5,13.96
2,EASTERN CHESHIRE CCG,01C00,MIDDLEWOOD PARTNERSHIP,N81022,0211000P0AAAAAA,23268711000001101,8,Tranexamic Acid_Inj 100mg/ml 5ml Amp,2,5,13.96
3,EASTERN CHESHIRE CCG,01C00,MIDDLEWOOD PARTNERSHIP,N81022,0211000P0AAAAAA,34756011000001103,8,Tranexamic Acid_Inj 100mg/ml 5ml Amp,2,5,13.96
4,EASTERN CHESHIRE CCG,01C00,MIDDLEWOOD PARTNERSHIP,N81022,0211000P0AAAAAA,35750911000001105,8,Tranexamic Acid_Inj 100mg/ml 5ml Amp,2,5,13.96


In [4]:
#monthly total
df_amp.sum()[['Items','Actual_Cost']]

Items              1568
Actual_Cost   115078.41
dtype: object

In [5]:
##groupby bnf name  to see "biggest offenders" in terms of items
df_amp_product = df_amp.groupby(['id', 'bnf_name', 'avail_restrict']).sum().sort_values(by = 'Items', ascending = False)
df_amp_product

,,,Items,Quantity,Actual_Cost
id,bnf_name,avail_restrict,,,
17652211000001105,"Colistimeth Sod_Inj 1,000,000u Vl",8,385,20240,39070.73
27953111000001108,Midazolam HCl_Inj 2mg/ml 5ml Amp,8,178,1317,1069.17
22786211000001105,Tranexamic Acid_Inj 100mg/ml 5ml Amp,8,94,1533,2712.20
22451411000001103,Tranexamic Acid_Inj 100mg/ml 5ml Amp,8,94,1533,2712.20
23268711000001101,Tranexamic Acid_Inj 100mg/ml 5ml Amp,8,94,1533,2712.20
35750911000001105,Tranexamic Acid_Inj 100mg/ml 5ml Amp,8,94,1533,2712.20
34756011000001103,Tranexamic Acid_Inj 100mg/ml 5ml Amp,8,94,1533,2712.20
11260411000001104,Viridal Duo_S/Pack Inj 10mcg Cart,8,62,79,1628.61
3998211000001106,Gentamicin Sulf_Inj 40mg/ml 2ml Vl,8,44,872,1734.80


In [6]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  Practice_Name, 
  Practice_Code,
  presc.bnf_code,
  dmdone.id,
  dmdtwo.hosp,
  bnf_description AS bnf_name,
  Items,
  Quantity,
  Actual_Cost
FROM
  ebmdatalab.tmp_eu.raw_prescribing_data_2019_04 AS presc ##this is raw quantity file. using this to see if there are patterns of small quantities
Left JOIN  
  ebmdatalab.dmd.ampp AS dmdone ##needed to link ampp code to bnf code
ON
presc.bnf_code = dmdone.bnf_code
LEFT JOIN
ebmdatalab.dmd.prescribinfo AS dmdtwo ##this is where product prescribing info is held
ON
dmdone.id = dmdtwo.ampp 
WHERE
dmdtwo.hosp = true
'''

df_ampp = bq.cached_read(sql, csv_path='hospampp.csv')
df_ampp.rename(columns={'PCO_Code':'pct'}, inplace=True)
df_ampp.head(5)

,PCO_Name,pct,Practice_Name,Practice_Code,bnf_code,id,hosp,bnf_name,Items,Quantity,Actual_Cost
0,THURROCK CCG,07G00,DR ABELA T PRACTICE,F81113,0501070I0BCABAB,4658911000001108,True,"Promixin_Inj 1,000,000u Vl (Dry)",1,56,195.22
1,THURROCK CCG,07G00,COMMONWEALTH HEALTH CENTRE,F81206,1501041T0AAABAB,27953211000001102,True,Midazolam HCl_Inj 2mg/ml 5ml Amp,1,10,6.66
2,THURROCK CCG,07G00,COMMONWEALTH HEALTH CENTRE,F81206,1501041T0AAABAB,19711311000001103,True,Midazolam HCl_Inj 2mg/ml 5ml Amp,1,10,6.66
3,THURROCK CCG,07G00,DR YADAVA N PRACTICE,F81211,0501070I0AAABAB,17652311000001102,True,"Colistimeth Sod_Inj 1,000,000u Vl",1,56,93.76
4,THURROCK CCG,07G00,DELL MEDICAL CENTRE,F81219,0501021G0AAACAC,34751111000001106,True,Ceftriaxone Sod_Inj 1g Vl,1,3,26.71


In [7]:
#monthly total
df_ampp.sum()[['Items','Actual_Cost']]

Items             335685
Actual_Cost   3032677.99
dtype: object

In [8]:
##groupby chemical
df_ampp_product = df_ampp.groupby(['id', 'bnf_name', 'hosp']).sum().sort_values(by = 'Items', ascending = False)
df_ampp_product

,,,Items,Quantity,Actual_Cost
id,bnf_name,hosp,,,
17423911000001109,Laxido_Oral Pdr Sach (Orange) S/F,True,226420,2016364,1220049.09
21495711000001101,Seebri_Breezhaler Inh Cap 55mcg + Dev,True,31820,198420,921225.64
10844011000001107,Voltarol Emulgel_Aq Gel 1.16%,True,20748,898100,108171.11
19711511000001109,Midazolam HCl_Inj 5mg/ml 2ml Amp,True,15617,61705,79457.09
16614211000001109,Levest 150/30_Tab,True,13164,487312,35959.45
4840411000001107,Ventolin_Nebules Soln 2.5mg/2.5ml Ud,True,6490,262050,28980.13
2154811000001100,Nexium_Tab 20mg,True,3586,125631,104505.33
4852411000001106,Isotretinoin_Cap 20mg,True,3049,60005,121329.72
1919011000001101,Kapake_Tab 30mg/500mg,True,2721,241776,25255.98


A cursory look shows this AMPP list is very different from AMP list and appears to significant amount of non hospital Rx (e.g. laxido, Voltarol). The AMPP list consequently has a lot more items attrbuted to it. Explanations include
 - a) miscoded
 - b) AMPP does not neatly link to BNF code i.e. there is both a hosital and general pack available at AMPP which have both been mapped to a BNF code
 - c) Mapping provided by BSA is wrong

Example
17423911000001109	Laxido_Oral Pdr Sach (Orange) S/F	True	226420	2016364	1220049.09

The dmd code for this [product](https://openprescribing.net/dmd/ampp/17423911000001109/) which is a 4x2sachets pack size from Galen Ltd that has now been discontinued. It is possible that 
 -  this pack size was only available through hospitals
 -  b) above is correct and we can't accurately use this flag to map to prescribing
 - something else!